# <font color='blue'>1. User Details</font>

Name: Pravat Ranjan Pasayat

Master in Big Data and Business Analyitcs

IE University

May 2019

# <font color='blue'>2. Context</font>

Bike sharing systems are a new generation of traditional bike rentals where the whole process from membership, rental and return back has become automatic. Through these systems, user is able to easily rent a bike from a particular position and return back to another position. Currently, there are about over 500 bike-sharing programs around the world which are composed of over 500 thousands bicycles. Today, there exists great interest in these systems due to their important role in traffic, environmental and health issues.

Apart from interesting real-world applications of bike sharing systems, the characteristics of data being generated by these systems make them attractive for the research. Opposed to other transport services such as bus or subway, the duration of travel, departure and arrival position is explicitly recorded in these systems. This feature turns bike sharing system into a virtual sensor network that can be used for sensing mobility in the city. Hence, it is expected that most of important events in the city could be detected via monitoring these data.

This dataset contains the hourly and daily count of rental bikes between years 2011 and 2012 in Capital bikeshare system in Washington, DC with the corresponding weather and seasonal information.

# <font color='blue'>3. Content</font>

Both hour.csv and day.csv have the following fields, except hr which is not available in day.csv.

1. instant: Record index
2. dteday: Date
3. season: Season (1:springer, 2:summer, 3:fall, 4:winter)
4. yr: Year (0: 2011, 1:2012)
5. mnth: Month (1 to 12)
6. hr: Hour (0 to 23)
7. holiday: weather day is holiday or not (extracted from Holiday Schedule)
8. weekday: Day of the week
9. workingday: If day is neither weekend nor holiday is 1, otherwise is 0.
10. weathersit: (extracted from Freemeteo)     
    a: Clear, Few clouds, Partly cloudy, Partly cloudy         
    b: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist         
    c: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds         
    d: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog      
11. temp: Normalized temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-8, t_max=+39 (only in hourly scale)
12. atemp: Normalized feeling temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-16, t_max=+50 (only in hourly scale)
13. hum: Normalized humidity. The values are divided to 100 (max)
14. windspeed: Normalized wind speed. The values are divided to 67 (max)
15. casual: count of casual users
16. registered: count of registered users
17. cnt: count of total rental bikes including both casual and registered

# <font color='blue'>4. Load necessary libraries</font>

This section is only dedicated to load necessary Python library that are used later in the course of this task.

In [13]:
import numpy as np

from scipy.stats.stats import pearsonr
from math import sqrt
from scipy import stats

import dask.dataframe as dd
import dask.array as da
from dask_ml.datasets import make_regression
from dask_ml.preprocessing import DummyEncoder
from dask_ml.linear_model import LinearRegression
from dask_ml.xgboost import XGBRegressor
from dask.distributed import Client

from sklearn.metrics import r2_score
from sklearn.externals import joblib
from sklearn.externals.joblib import parallel_backend
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

# Supress warnings
warnings.filterwarnings("ignore")

# <font color='blue'>5. Read data</font>

1. The TRAIN and TEST data are both present in the file 'hour.csv'. 
2. The test set is the last quarter of the year 2012.
3. The rest of the data will be used for training.

In [10]:
# Load the data from a remote location to your working directory

!curl -O "https://raw.githubusercontent.com/pasayatpravat/Data_files/master/Bike_Sharing/hour.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 1112k    0  4674    0     0   4272      0  0:04:26  0:00:01  0:04:25  4272
100 1112k  100 1112k    0     0   878k      0  0:00:01  0:00:01 --:--:--  878k


In [12]:
# Load data
dataset_ddf = dd.read_csv("hour.csv")

# Set index
dataset_ddf = dataset_ddf.set_index("instant")

# Create 10 partitions of the dataframe
dataset_ddf = dataset_ddf.repartition(npartitions=5)